# 京大のBERT日本語Pretrainedモデル(pytorch版)でmask言語モデルを試す
【参考】 https://qiita.com/masaki_sfc/items/1564cf9122db7ed47096

## model/tokenizerを初期化

In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForPreTraining, modeling
from pyknp import Juman

In [2]:
config = modeling.BertConfig(
    vocab_size_or_config_json_file=32006,
    hidden_size=768, 
    num_hidden_layers=12,
    num_attention_heads=12, 
    intermediate_size=3072
)
model = BertForPreTraining(config=config)

In [3]:
model.load_state_dict(torch.load("../models/Japanese_L-12_H-768_A-12_E-30_BPE/pytorch_model.bin"))
tokenizer = BertTokenizer("../models/Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt", do_lower_case=False)

In [4]:
jm = Juman()

## MASK文を準備

In [5]:
TEXT = "今日は良い天気でサッカーがしたくなりますね。"

In [6]:
res = jm.analysis(TEXT)

In [7]:
tokens = ["[CLS]"] + [i.midasi for i in res.mrph_list()]

In [8]:
tokens

['[CLS]',
 '今日',
 'は',
 '良い',
 '天気',
 'で',
 'サッカー',
 'が',
 'し',
 'たく',
 'なり',
 'ます',
 'ね',
 '。']

In [9]:
tokens[6] = "[MASK]"

In [10]:
tokens = tokenizer.tokenize(" ".join(tokens))
ids = tokenizer.convert_tokens_to_ids(tokens)

## MASK文を確認

In [11]:
print("#   id     token")
for i, j in enumerate(tokens):
    print("{:2d}  {:<5d}  {}".format(i, ids[i], j))

#   id     token
 0  2      [CLS]
 1  2281   今日
 2  9      は
 3  2421   良い
 4  9292   天気
 5  13     で
 6  4      [MASK]
 7  11     が
 8  31     し
 9  5828   たく
10  105    なり
11  1953   ます
12  2382   ね
13  7      。


## MASK部分を推測

In [12]:
ids = torch.tensor(ids).reshape(1,-1)
model.eval()
with torch.no_grad():
    output, _ = model(ids)

In [13]:
MASK_POS = 6
print(" ".join(tokens))
print()
print("rank   スコア   token")
for i, j in enumerate(output[0][MASK_POS].argsort(descending=True)[:20]):
    print(" {:2d}   {:6.4f}  {}".format(i+1, output[0][MASK_POS][j].item(), tokenizer.ids_to_tokens[j.item()]))

[CLS] 今日 は 良い 天気 で [MASK] が し たく なり ます ね 。

rank   スコア   token
  1   8.7932  仕事
  2   7.8080  話
  3   7.1951  ゲーム
  4   6.6053  食事
  5   6.4928  勉強
  6   6.4125  試合
  7   6.2921  旅
  8   6.1778  歌
  9   6.0824  買い物
 10   6.0327  サーフィン
 11   5.9291  遊び
 12   5.7836  レース
 13   5.7719  ライブ
 14   5.5982  掃除
 15   5.5943  レコーディング
 16   5.5247  練習
 17   5.0662  ドラマ
 18   5.0646  生活
 19   5.0603  [UNK]
 20   5.0379  議論
